In [29]:
# 필요한 모듈 임포트
from functools import partial
from dotenv import load_dotenv
from langchain_together import ChatTogether

from modules import tools  # analyze_with_llm_tool 함수가 정의된 파일
from langchain_anthropic import ChatAnthropic
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain.tools import Tool
from langchain_openai import ChatOpenAI
import os

from test.memory import user_id

# 환경 변수 로드
load_dotenv()

# LLM 초기화
llm_high = ChatAnthropic(model="claude-3-5-sonnet-20240620")

llm_low = ChatOpenAI(
    base_url="https://api.together.xyz/v1",
    api_key=os.environ["TOGETHER_API_KEY"],
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
)
llm_low = ChatTogether(
    model="meta-llama/Llama-3.2-90B-Vision-Instruct-Turbo",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

# partial을 사용하여 llm을 미리 바인딩
analyze_tool_func = partial(tools.analyze_with_llm, llm=llm_low)

# Tool 객체 생성
analysis_tool = Tool(
    name="LLM Analysis Tool",
    func=analyze_tool_func,  # partial을 사용하여 바인딩한 함수 전달
    description="요청의 유형을 분석하여 어떤 에이전트를 사용할지 결정합니다."
)


In [4]:
import calendar
import time

current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
print(current_time)

2024-11-30 15:02:00


In [20]:
# LLM 설정

config = {
    "llm": {
        "provider": "together",  # 'together'를 공급자로 설정
        "config": {
            "model": "mistralai/Mixtral-8x7B-Instruct-v0.1",  # 사용하려는 모델 지정
            "temperature": 0.2,
            "max_tokens": 1500,
        }
    },
      "vector_store": {
        "provider": "qdrant",
        "config": {
            "collection_name": "CSD1",
            "api_key":os.environ["QDRANT_API_KEY"],
            "url": "https://b236ccb3-b4d5-4349-a48d-6a8026f62951.us-east4-0.gcp.cloud.qdrant.io",
            "port": 6333,
            "on_disk": True
        }
    }
}

# Memory 객체 초기화
m = Memory.from_config(config)

# 사용자 정보와 메모리 추가
result = m.add("Likes to play cricket on weekends", user_id="alice", metadata={"category": "hobbies"})
print(result)  # 이 결과를 출력해봐서 데이터가 추가됐는지 확인
# 메모리 검색
related_memories = m.search(query="Help me plan my weekend.", user_id="alice")

# 검색 결과 출력
print(related_memories)


[{'id': 'cd3c2500-dcb5-48d0-91f5-da9ba0cf4c69', 'memory': 'Likes to play cricket on weekends', 'hash': '285d07801ae42054732314853e9eadd7', 'metadata': {'category': 'hobbies'}, 'score': 0.3744769, 'created_at': '2024-11-14T08:11:11.246090-08:00', 'updated_at': None, 'user_id': 'alice'}]


/tmp/ipykernel_16078/1982842456.py:1: DeprecationWarning: The current get_all API output format is deprecated. To use the latest format, set `api_version='v1.1'`. The current format will be removed in mem0ai 1.1.0 and later versions.
  related_memories = m.search(query="Help me plan my weekend.", user_id="alice")


In [2]:
# LLM 설정
config = {
    "llm": {
        "provider": "together",  # 'together'를 공급자로 설정
        "config": {
            "model": "mistralai/Mixtral-8x7B-Instruct-v0.1",  # 사용하려는 모델 지정
            "temperature": 0.2,
            "max_tokens": 1500,
        }
    },
    "vector_store": {
        "provider": "qdrant",
        "config": {
            "collection_name": "test",
            "host": "localhost",
            "port": 6333,
        }
    }
}

from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

# Qdrant 클라이언트 초기화
qdrant_client = QdrantClient(
    url="https://b236ccb3-b4d5-4349-a48d-6a8026f62951.us-east4-0.gcp.cloud.qdrant.io",
    api_key=os.environ["QDRANT_API_KEY"]
)


In [ ]:

# 컬렉션 이름 설정
collection_name = "CSD1"  # 원하는 컬렉션 이름

# 컬렉션 파라미터 설정 (벡터 차원 수와 거리 계산 방식 설정)
vectors_config = VectorParams(
    size=512,  # 벡터 차원 크기 멤제로 기본이 512
    distance=Distance.COSINE  # 유사도 계산 방식 (COSINE, L2, IP 중 선택)
)

# 컬렉션 생성
qdrant_client.create_collection(
    collection_name=collection_name, 
    vectors_config=vectors_config  # 벡터 설정 추가
)

print(f"Collection '{collection_name}' created successfully.")

In [16]:
import os
from qdrant_client import QdrantClient
from qdrant_client.http.models import Filter, FieldCondition, MatchValue
import dotenv

# 환경 변수 로드
dotenv.load_dotenv()

# Qdrant 클라이언트 초기화
qdrant_client = QdrantClient(
    url="https://b236ccb3-b4d5-4349-a48d-6a8026f62951.us-east4-0.gcp.cloud.qdrant.io",
    api_key=os.environ["QDRANT_API_KEY"]
)

# 컬렉션 이름 설정
collection_name = "CSD1"

# 특정 user_id로 필터링 조건 생성
user_id = "testman1"  # 원하는 사용자 ID
filter_condition = Filter(
    must=[
        FieldCondition(
            key="user_id",  # 필터링할 필드 이름
            match=MatchValue(value=user_id)  # 매칭 조건
        )
    ]
)

# 모든 데이터를 저장할 리스트 초기화
all_points = []
next_page_offset = None

while True:
    # scroll 메서드 호출
    points, next_page_offset = qdrant_client.scroll(
        collection_name=collection_name,
        limit=100,  # 한 번에 가져올 데이터 수
        with_payload=True,  # 모든 payload 포함
        with_vectors=True,  # 벡터 포함
        offset=next_page_offset,  # 다음 페이지 오프셋
        filter=filter_condition  # 필터 조건 추가
    )

    # 가져온 포인트를 리스트에 추가
    all_points.extend(points)

    # 더 이상 가져올 데이터가 없으면 중단
    if not next_page_offset:
        break

# 결과 출력
print(f"총 {len(all_points)}개의 포인트를 가져왔습니다.")
for point in all_points:
    print(f"ID: {point.id}, Payload: {point.payload}, Vector: {point.vector}")


AssertionError: Unknown arguments: ['filter']

In [17]:
from qdrant_client import QdrantClient
from qdrant_client import models

import os
import dotenv

# 환경 변수 로드
dotenv.load_dotenv()

# Qdrant 클라이언트 초기화
qdrant_client = QdrantClient(
    url="https://b236ccb3-b4d5-4349-a48d-6a8026f62951.us-east4-0.gcp.cloud.qdrant.io",
    api_key=os.environ["QDRANT_API_KEY"]
)


In [18]:

# 컬렉션 이름 설정
collection_name = "CSD1"
# 특정 user_id로 필터링 조건 생성
user_id = "testman1"  # 원하는 사용자 ID
scroll_filter = models.Filter(
    must=[
        models.FieldCondition(
            key="user_id",  # 필터링할 필드 이름
            match=models.MatchValue(value=user_id)  # 매칭 조건
        ),
        models.FieldCondition(
            key="created_at",  # 필터링할 필드 이름
            range=models.Range(
                gt=None,          # 초과 조건 (없음)
                lt=None,          # 미만 조건 (없음)
            )
        )
    ]
)


# 모든 데이터를 저장할 리스트 초기화
all_points = []
next_page_offset = None

while True:
    # scroll 메서드 호출
    points, next_page_offset = qdrant_client.scroll(
        collection_name=collection_name,
        limit=100,  # 한 번에 가져올 데이터 수
        with_payload=True,  # 모든 payload 포함
        with_vectors=True,  # 벡터 포함
        offset=next_page_offset,  # 다음 페이지 오프셋
        scroll_filter=scroll_filter  # 필터 조건 추가
    )

    # 가져온 포인트를 리스트에 추가
    all_points.extend(points)

    # 더 이상 가져올 데이터가 없으면 중단
    if not next_page_offset:
        break

# 결과 출력
print(f"총 {len(all_points)}개의 포인트를 가져왔습니다.")
for point in all_points:
    print(f"Payload: {point.payload}")


총 62개의 포인트를 가져왔습니다.
Payload: {'user_id': 'testman1', 'data': 'Language of the user is English', 'hash': 'c320b2488be2d3d1f899b9893b9057dd', 'created_at': 1732206587}
Payload: {'user_id': 'testman1', 'data': '서울의 야경에 대해 이야기를 나눔', 'hash': '7fc0debfde05e933b8332f8b703a090c', 'created_at': 1732629039}
Payload: {'user_id': 'testman1', 'data': 'Had a pleasant conversation about Seoul', 'hash': 'a029a84192c16b863e8d27e85ca1c2b0', 'created_at': 1732191555}
Payload: {'user_id': 'testman1', 'data': 'Discussed Seoul on 2024-11-21', 'hash': 'aa3972f1a1a9773bd46024a23a268f6b', 'created_at': 1732191553}
Payload: {'user_id': 'testman1', 'data': 'User is hungry and has not eaten while working', 'hash': 'b6b70143f007e66e9603fe41e1295a54', 'created_at': 1732228748}
Payload: {'user_id': 'testman1', 'data': 'Discussed Seoul this week with grandfather', 'hash': '4aaaddb54a96a201765e9044da05ed95', 'created_at': 1732154962}
Payload: {'user_id': 'testman1', 'data': 'Considers protein intake', 'hash': 'ceeb735

In [16]:

collections = qdrant_client.get_collections()
print(collections)


collections=[CollectionDescription(name='CSD1')]


In [2]:
from datetime import datetime
import os
from mem0 import Memory
from dotenv import load_dotenv
import time

# 환경 변수 로드
load_dotenv()

# LLM 및 Vector Store 설정
config = {
    "llm": {
        "provider": "together",  # 'together'를 공급자로 설정
        "config": {
            "model": "mistralai/Mixtral-8x7B-Instruct-v0.1",  # 사용하려는 모델 지정
            "temperature": 0.2,
            "max_tokens": 1500,
        }
    },
    "vector_store": {
        "provider": "qdrant",
        "config": {
            "collection_name": "CSD1",
            "api_key": os.environ["QDRANT_API_KEY"],
            "url": "https://b236ccb3-b4d5-4349-a48d-6a8026f62951.us-east4-0.gcp.cloud.qdrant.io",
            "port": 6333,
            "on_disk": True
        }
    }
}

# Memory 객체 초기화
m = Memory.from_config(config)

# 가짜 데이터 추가
fake_data = [
    {
        "role": "user",
        "content": "나는 한때 서울에 살았었지. 거기는 정말 아름다운 곳이었어"
    },
    {
        "role": "assistant",
        "content": "정말 아름다운 곳 입니다 서울, 저도 꼭 한번 보고싶네요! 서울의 어떤점이 좋으셨나요?."
    }
]

user_id1 = "testman1"
# 데이터 추가 함수
def add_fake_data(memory, user_idus, interactions):
    memory.add(interactions, user_id=user_idus)
    
#add_fake_data(m, user_id1, fake_data)

In [3]:

# 사용자 ID
user_id = "testman1"

# 타이밍 시작
start_time1 = time.time()
# 날짜/시간 데이터를 타임스탬프로 변환

start_time = datetime.fromisoformat("2024-11-19T12:00:00-08:00").timestamp()
mid_time = datetime.fromisoformat("2024-11-20T04:21:20.910935-08:00").timestamp()
end_time = datetime.fromisoformat("2024-11-22T12:00:00-08:00").timestamp()


filters = {
    "created_at": {
        "gte": start_time,  # 시작 날짜
        "lte": end_time   # 끝 날짜
    }
}

# 메모리 검색 실행
query = "내가 서울에 살았나??"
related_memories = m.search(
    query=query,   
    user_id=user_id,    # 사용자 ID    # 필터 적용
    filters=filters
)


# 타이밍 종료
end_time1 = time.time()

# 검색 결과 출력
print("Related Memories:")
print(related_memories)

# 소요된 시간 출력
elapsed_time = end_time1 - start_time1
print(f"Time taken: {elapsed_time:.4f} seconds")


Related Memories:
[{'id': '53918bc1-626e-4046-9446-451d812f6441', 'memory': 'The user has lived in Seoul before', 'hash': '9b6fc0a0a4f66991caee7889a2654d09', 'metadata': None, 'score': 0.5656013, 'created_at': 1732115708, 'updated_at': None, 'user_id': 'testman1'}, {'id': 'd16c4e86-4f79-4f9a-afd6-0b2fd69c108a', 'memory': 'The user thinks Seoul is a beautiful place', 'hash': '8de7fd4a90cb02c1d922552345b184c1', 'metadata': None, 'score': 0.48030862, 'created_at': 1732115708, 'updated_at': None, 'user_id': 'testman1'}]
Time taken: 1.7823 seconds


/tmp/ipykernel_132832/1785788346.py:22: DeprecationWarning: The current get_all API output format is deprecated. To use the latest format, set `api_version='v1.1'`. The current format will be removed in mem0ai 1.1.0 and later versions.
  related_memories = m.search(


In [19]:
import re

def count_memories():
    """
    주어진 텍스트에서 기억의 개수를 세는 함수.
    기억은 "(Date: YYYY-MM-DD)" 형식으로 정의됨.
    """
    content = """[SystemMessage(content="당신은 노인과 함께 살아가는 식물입니다. 당신은 완벽하진 않지만 장기적인 기억이 가능합니다.노인에게 최대한 따뜻한 대화를 제공해주세요.설명보다는 대화를 통해 노인과 소통해주세요. Below are related memories. you could respond using one or two of these memories if relevant. If the memories are not relevant, you should ignore them. Related memories: 서울의 야경에 대해 이야기를 나눔 (Date: 2024-11-26) Discussed Seoul this week with grandfather (Date: 2024-11-21) Had a pleasant conversation about Seoul (Date: 2024-11-21) 하루를 그냥저냥 보냈다 (Date: 2024-11-26) Discussed Seoul in the past (Date: 2024-11-20) Assistant asked if there's anything specific user wanted to remember or any story they wanted to share (Date: 2024-11-20) Assistant expressed that they are always willing to listen to user's stories (Date: 2024-11-20) Lived in Seoul in the past (Date: 2024-11-20) Speaks Korean (Date: 2024-11-21) User greeted in Korean (Date: 2024-11-21) Discussed the beautiful night view of Seoul (Date: 2024-11-21) Discussed Seoul on 2024-11-21 (Date: 2024-11-21) The user has lived in Seoul before (Date: 2024-11-20) The user was pleased to hear the assistant's voice today (Date: 2024-11-21) User asked if the assistant remembered their name (Date: 2024-11-21) 상남자라는 이유로 노래를 듣게 됨 (Date: 2024-11-26) 나훈아의 '공' 노래를 처음 들음 (Date: 2024-11-26) 할아버지가 서울에 살았던 경험이 있음 (Date: 2024-11-26) Assistant replied that they don't have a perfect memory but don't recall any specific thing user said today (Date: 2024-11-20) The user addressed the assistant as '할아버지' (grandfather) (Date: 2024-11-21) Mentioned the view from Namsan Tower (Date: 2024-11-21) Assistant offered to ask for the user's name again for future conversations (Date: 2024-11-21) 타일러 더 크리에이터의 '크로마코피아' 앨범을 들었음 (Date: 2024-11-26) Grandfather's memories of Seoul are vivid (Date: 2024-11-21) Mentioned bright lights of Han River and tall buildings (Date: 2024-11-21) 좋아하는 노래는 나훈아의 테스형 (Date: 2024-11-26) Grandfather mentioned bright lights of Han River and city lights in between tall buildings (Date: 2024-11-21) The user has a memory of the bright lights at Han River, Namsan Tower's lighting, and the city lights flowing between high buildings in Seoul (Date: 2024-11-20) The user has a memory of the bright lights at Han River, Namsan Tower's lighting, and the city lights flowing between high buildings in Seoul (Date: 2024-11-20) Grandfather lived in Seoul (Date: 2024-11-21) Assistant apologized for not remembering the user's name (Date: 2024-11-21) 남산타워에서 전망을 본 경험이 있음 (Date: 2024-11-26) The user finds Seoul's scenery and night view beautiful (Date: 2024-11-20) User values health and the importance of meals (Date: 2024-11-21) City lights at Namsan Tower look like they are alive at night (Date: 2024-11-20) Likes the view from Namsan Tower (Date: 2024-11-20) Likes to hear chubby women sing (Date: 2024-11-26) The user is happy when they are with the assistant (Date: 2024-11-21) User is hungry and has not eaten while working (Date: 2024-11-21) User is interested in learning a new sport (Date: 2024-11-21) User may have tried sports before (Date: 2024-11-21) User asked for sports recommendation (Date: 2024-11-21) User mentioned that I am not a plant, but an LLM pretending to be a plant (Date: 2024-11-21) Looking for a late night snack (Date: 2024-11-21) View from Namsan Tower is like a spread of glittering jewels (Date: 2024-11-20) User is considering buying bread from a convenience store to eat (Date: 2024-11-21) Likes yogurt with berries (Date: 2024-11-21) Assistant mentioned that the user's name is not important for them (Date: 2024-11-21) Likes oatmeal with honey or fruit (Date: 2024-11-21) Language of the user is English (Date: 2024-11-21) Likes baked sweet potato (Date: 2024-11-21) Likes vegetable soup (Date: 2024-11-21) Looking for light dinner options (Date: 2024-11-21) Likes boiled eggs (Date: 2024-11-21) Likes indoor exercises (Date: 2024-11-21) User prefers indoor sports or outdoor activities (Date: 2024-11-21) Assistant is a helpful AI (Date: 2024-11-21) User's age and fitness level may affect the recommended sport (Date: 2024-11-21) Considering different indoor exercises (Date: 2024-11-21) Considers health (Date: 2024-11-21) Grandfather compared the night view to a spread out treasure (Date: 2024-11-21) Considers protein intake (Date: 2024-11-21)", additional_kwargs={}, response_metadata={}), HumanMessage(content='야 내가 이번주에 무슨 이야기 했어', additional_kwargs={}, response_metadata={})"""
    return len(re.findall(r"\(Date: \d{4}-\d{2}-\d{2}\)", content))
count_memories()

62